# Project 3: User Activity

## Week 11:

For Week 11, we used the following linux commands to complete the tasks: 

`cp ~/w205/course-content/11-Storing-Data-III/game_api.py ~/w205/project-3-daphne-yang`

`docker-compose up -d`
`docker ps -a`

`docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181`


`docker-compose exec mids env FLASK_APP=/w205/project-3-daphne-yang/game_api.py flask run --host 0.0.0.0`

In a new window: 

`cd ~/w205/project-3-daphne-yang`

`docker-compose exec mids curl  http://localhost:5000/`
`docker-compose exec mids curl   http://localhost:5000/purchase_a_sword`
`docker-compose exec mids curl  http://localhost:5000/buy_a_sword`
`docker-compose exec mids curl  http://localhost:5000/join_a_guild`

`docker-compose exec spark bash`
`ln -s /w205 w205`
`exit`

`docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark`

`docker-compose down`

`docker ps -a`

In [9]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf

In [14]:
@udf('string')
def munge_event(event_as_json):
    event = json.loads(event_as_json)
    event['Host'] = "moe"
    event['Cache-Control'] = "no-cache"
    return json.dumps(event)

In [15]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [16]:
raw_events.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 48 6F 73 7...|events|        0|     0|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     1|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     2|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     3|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     4|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     5|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     6|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0|     7|2020-12-06 02:33:...|            0|
|null|[7B 22 48 6F 73 7...|events|        0

In [17]:
munged_events = raw_events \
    .select(raw_events.value.cast('string').alias('raw'),
            raw_events.timestamp.cast('string')) \
    .withColumn('munged', munge_event('raw'))

In [22]:
munged_events.show()

+--------------------+--------------------+--------------------+
|                 raw|           timestamp|              munged|
+--------------------+--------------------+--------------------+
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|2020-12-06 02:33:...|{"Host": "moe", "...|
|{"Host": "user1.c...|202

In [18]:
extracted_events = munged_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.munged))) \
        .toDF()

In [23]:
extracted_events.show()

Py4JJavaError: An error occurred while calling o127.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:235)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPar

In [24]:
sword_purchases = extracted_events \
        .filter(extracted_events.event_type == 'purchase_sword')

In [25]:
sword_purchases.show()

Py4JJavaError: An error occurred while calling o188.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 7 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$.fromJava(EvaluatePython.scala:138)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at org.apache.spark.sql.SparkSession$$anonfun$5.apply(SparkSession.scala:728)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apa

In [26]:
default_hits = extracted_events \
        .filter(extracted_events.event_type == 'default')

In [27]:
default_hits

DataFrame[Accept: string, Cache-Control: string, Host: string, User-Agent: string, event_type: string, timestamp: string]

## Week 12:

#### For Week 12, we used the following linux commands to complete the tasks: 

`cp ~/w205/course-content/12-Querying-Data-II/docker-compose.yml ~/w205/project-3-daphne-yang`

`docker-compose up -d`
`docker-compose ps`
`docker ps -a`

We don’t need to copy in game_api.py again because we already did that in Wk 11. We need to edit and add in meta data.
* Added in sword length, guild name, and sword type knight

`docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181`


`docker-compose exec mids env FLASK_APP=/w205/project-3-daphne-yang/game_api.py flask run --host 0.0.0.0`
* This blocks out the window

In a new window: 

`cd ~/w205/project-3-daphne-yang`

`docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com"  http://localhost:5000/`
`docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword`
`docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/buy_a_sword`
`docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/join_a_guild`

`docker-compose exec mids ab -n 10 -H "Host: user2.att.com"  http://localhost:5000/`
`docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword`
`docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/buy_a_sword`
`docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/join_a_guild`

`docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e`


`docker-compose exec spark bash`
`ln -s /w205 w205`
`exit`

`docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark`


`docker-compose down`

`docker ps -a`

This is code from `filtered_write.py`

In [19]:
import json
from pyspark.sql import Row
from pyspark.sql.functions import udf

In [28]:
@udf('boolean')
def is_purchase(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [29]:
raw_events = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [30]:
purchase_events = raw_events \
        .select(raw_events.value.cast('string').alias('raw'),
                raw_events.timestamp.cast('string')) \
        .filter(is_purchase('raw'))

In [31]:
extracted_purchase_events = purchase_events \
        .rdd \
        .map(lambda r: Row(timestamp=r.timestamp, **json.loads(r.raw))) \
        .toDF()

In [32]:
extracted_purchase_events.printSchema()

root
 |-- Accept: string (nullable = true)
 |-- Host: string (nullable = true)
 |-- User-Agent: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- sword_type: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [33]:
extracted_purchase_events.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|us

In [35]:
extracted_purchase_events \
        .write \
        .mode('overwrite') \
        .parquet('/tmp/purchases')

## Pyspark Code For Wk 12

In [36]:
purchases = spark.read.parquet('/tmp/purchases')

In [37]:
purchases.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|us

In [38]:
purchases.registerTempTable('purchases')

In [39]:
purchases_by_example2 = spark.sql("select * from purchases where Host = 'user1.comcast.com'")

In [40]:
purchases_by_example2.show()

+------+-----------------+---------------+--------------+----------+--------------------+
|Accept|             Host|     User-Agent|    event_type|sword_type|           timestamp|
+------+-----------------+---------------+--------------+----------+--------------------+
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|user1.comcast.com|ApacheBench/2.3|purchase_sword|   knights|2020-12-06 02:33:...|
|   */*|us

In [41]:
df = purchases_by_example2.toPandas()

In [43]:
df

,Accept,Host,User-Agent,event_type,sword_type,timestamp
0,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.29
1,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.294
2,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.297
3,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.3
4,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.302
5,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.305
6,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.308
7,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.31
8,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.314
9,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.316


In [42]:
df.describe()

,Accept,Host,User-Agent,event_type,sword_type,timestamp
count,10,10,10,10,10,10
unique,1,1,1,1,1,10
top,*/*,user1.comcast.com,ApacheBench/2.3,purchase_sword,knights,2020-12-06 02:33:52.316
freq,10,10,10,10,10,1


### Simple Analytics of DataFrame Using Spark SQL

#### <i>Question 1:</i> 
How many User-Agents purchased a sword?

In [52]:
sword_purchases = spark.sql("select count(*) as Number_User_Agent from purchases where event_type = 'purchase_sword'")
sword_purchases.show()

+-----------------+
|Number_User_Agent|
+-----------------+
|               20|
+-----------------+



#### <i>Question 2:</i> 
How many User-Agents are using att.com as a host?

In [50]:
user_hosts = spark.sql("select count(*) as Number_Hosts from purchases where Host = 'user2.att.com'")
user_hosts.show()

+------------+
|Number_Hosts|
+------------+
|          10|
+------------+



## Week 13:

#### For Week 13, we used the following linux commands to complete the tasks: 

`cp ~/w205/course-content/13-Understanding-Data/docker-compose.yml ~/w205/project-3-daphne-yang`

`vi docker-compose.yml`
- Uncomment out the spark ports


`docker-compose up -d`
`docker-compose ps`
`docker ps -a`

We don’t need to copy in game_api.py again because we already did that in Wk 12. Can use as is 

`docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181`

`docker-compose exec mids env FLASK_APP=/w205/project-3-daphne-yang/game_api.py flask run --host 0.0.0.0`
* This blocks out the window

In Wk 13, we will be using infinite loop streaming instead of single commands using the following Apache Bench Command: 

`while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword; sleep 5; done`


In a new window: 

`docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e`


`docker-compose exec spark bash`
`ln -s /w205 w205`
`exit`

`docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark`

`docker-compose exec cloudera hadoop fs -ls /tmp/sword_purchases`

`docker-compose exec cloudera hive`

To create tables:
`create external table if not exists default.sword_purchases (Accept string, Host string, User_Agent string, event_type string, timestamp string) stored as parquet location '/tmp/sword_purchases'  tblproperties ("parquet.compress"="SNAPPY");`

In Hive - Check for tables:

`show tables;`

`exit;`


Query from presto:

`docker-compose exec presto presto --server presto:8080 --catalog hive --schema default`

In presto:

`show tables;`

`select count(*) from sword_purchases;`

Shutting down clusters:

`docker-compose down`

`docker ps -a`

From `write_swords_stream.py`

In [4]:
import json
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [5]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

In [6]:
@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False

In [7]:
raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

In [8]:
sword_purchases = raw_events \
        .filter(is_sword_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

In [11]:
sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="120 seconds") \
        .start()

In [10]:
sink.stop()